In [1]:
import os
import shutil
import time
import torch


#
#The idea is to merge sequences recursively. FILM creates frames set between 0000 frame of first sequence 
#and 0000 of second sequence, and between 0002 frame of first sequence and 0002 of second sequence, 
#then code picks frame 0 from first frame set and frame 2 from second frames set and interpolates between 
#them to get final frame 0001. Then there is interpolation between resulting frame 0001 and frame 3 from interpolating 
#between frames 0003 of first and second sequence to get final frame 0002 and so on...  Some names in code remained from 
#my experimentation with Rife, it's uses less resourses but FILM gives better quality. 
#

#--------------works only if there's equal distance between keyframes e.g. that can be 0000 0015 0030 frames and so on


sequences_folder = "Stash" # Folder containing sequences 
fade_duration = 16 # Number of n+1 keyframe minus number of n keyframe plus 1, following logic of sequences output of EbSynth

# press_to_continue = True  

model_path = 'film_net_fp32.pt'

gpu='gpu' #gpu or cpu
half = 'nohalf' #precision of model

# Output folder for the crossfaded images
base_rife_folder = "FILMoutput"

inter_frames = fade_duration - 2 #Following logic of FILM interpolaion

inter_frames2 = 1

# Output folder for the crossfaded images
output_folder = sequences_folder + f'_stitched_double_film_I_._I_recursive'

if os.path.exists(base_rife_folder):
    shutil.rmtree(base_rife_folder)
os.makedirs(base_rife_folder, exist_ok=True)

os.makedirs(output_folder, exist_ok=True)

# Paths to your input image sequences
sequences = os.listdir(sequences_folder)

if torch.cuda.is_available(): #gpu check
    current_device = torch.cuda.current_device()
    device_name = torch.cuda.get_device_name(current_device)
    print(f"Using GPU: {device_name}")

print('Using sequences:')
print('')
total_sequence_length = 0
for i in range(len(sequences)):
    sequence_length = len(os.listdir(os.path.join(sequences_folder, sequences[i])))
    
    total_sequence_length += sequence_length
    print(i, sequences[i], 'number of frames:', sequence_length)
print('')
print('Total number of frames:', total_sequence_length)
print('')

counter = 0

filename = os.listdir(sequences_folder)[0]

base_name, file_extension = os.path.splitext(filename)

parts = base_name.split("_")

counter = int(parts[1])-int(parts[1])//fade_duration

print('Counter set to', counter)



#-------------------------------------------Load model----------------------------------------    
    
import bisect
import numpy as np
import cv2

from util import load_image


def inference(model_path, img1, img2, save_path, gpu, inter_frames, fps, half):
    model = torch.jit.load(model_path, map_location='cpu')
    model.eval()
    img_batch_1, crop_region_1 = load_image(img1)
    img_batch_2, crop_region_2 = load_image(img2)

    img_batch_1 = torch.from_numpy(img_batch_1).permute(0, 3, 1, 2)
    img_batch_2 = torch.from_numpy(img_batch_2).permute(0, 3, 1, 2)

    if not half:
        model.float()

    if gpu and torch.cuda.is_available():
        
        if half:
            model = model.half()
        else:
            model.float()
        model = model.cuda()

    results = [
        img_batch_1,
        img_batch_2
    ]

    idxes = [0, inter_frames + 1]
    remains = list(range(1, inter_frames + 1))

    splits = torch.linspace(0, 1, inter_frames + 2)

    for _ in range(len(remains)):
        starts = splits[idxes[:-1]]
        ends = splits[idxes[1:]]
        distances = ((splits[None, remains] - starts[:, None]) / (ends[:, None] - starts[:, None]) - .5).abs()
        matrix = torch.argmin(distances).item()
        start_i, step = np.unravel_index(matrix, distances.shape)
        end_i = start_i + 1

        x0 = results[start_i]
        x1 = results[end_i]

        if gpu and torch.cuda.is_available():
            if half:
                x0 = x0.half()
                x1 = x1.half()
            x0 = x0.cuda()
            x1 = x1.cuda()

        dt = x0.new_full((1, 1), (splits[remains[step]] - splits[idxes[start_i]])) / (splits[idxes[end_i]] - splits[idxes[start_i]])

        with torch.no_grad():
            prediction = model(x0, x1, dt)
        insert_position = bisect.bisect_left(idxes, remains[step])
        idxes.insert(insert_position, remains[step])
        results.insert(insert_position, prediction.clamp(0, 1).cpu().float())
        del remains[step]

        os.makedirs(save_path, exist_ok=True)

        y1, x1, y2, x2 = crop_region_1
        frames = [(tensor[0] * 255).byte().flip(0).permute(1, 2, 0).numpy()[y1:y2, x1:x2].copy() for tensor in results]

        for i, frame in enumerate(frames):
            image_path = os.path.join(save_path, f"{i:05d}.png")
            cv2.imwrite(image_path, frame)
        
#-------------------------------------------Interpolation----------------------------------------         


user_input = 0

t0 = time.time()

counterk = 0

frames_left = total_sequence_length

for sequence_number in range(0, len(sequences)):  
    
    tseq_start = time.time()
    
    if sequence_number != len(sequences)-1:
        
        print('')
        
        print('------------------------------------------------------------------------------')
        
        print('')
        
        print('Sequences:', sequences[sequence_number], sequences[sequence_number+1] )
        
        print('')
        
        print('------------------------------------------------------------------------------')
        
        print('First go')
        
        print('------------------------------------------------------------------------------')
        
        print('counter:   ', counter)
        
        print('sequence count number:', sequence_number,'     ', sequence_number +1)
        
        print('')

        files_list_1 = os.listdir(os.path.join(sequences_folder, sequences[sequence_number]))
        sequence1_selected_images = files_list_1[-fade_duration:]
                
        files_list_2 = os.listdir(os.path.join(sequences_folder, sequences[sequence_number+1]))
        sequence2_selected_images = files_list_2
        
        for i in range(fade_duration): 
            
            output_init_folder = f'{base_rife_folder}/init/{sequences[sequence_number]}_{sequences[sequence_number+1]}/pair{counter:05d}'
            os.makedirs(output_init_folder, exist_ok=True)
            
            output_rife_folder = f'{base_rife_folder}/rife/{sequences[sequence_number]}_{sequences[sequence_number+1]}/pair{counter:05d}'
            os.makedirs(output_rife_folder, exist_ok=True)
                       
            shutil.copy(os.path.join(sequences_folder, sequences[sequence_number], sequence1_selected_images[i]), 
                        os.path.join (output_init_folder, sequence1_selected_images[i]))
            
            print('Using file: ', os.path.join(sequences_folder, sequences[sequence_number], sequence1_selected_images[i]))

            base_name, ext = os.path.splitext(sequence2_selected_images[i])
            new_filename = f"{base_name}_2{ext}"

            shutil.copy(os.path.join(sequences_folder, sequences[sequence_number+1], sequence2_selected_images[i]), 
                        os.path.join (output_init_folder, new_filename))
            
            print('Using file: ', os.path.join(sequences_folder, sequences[sequence_number+1], sequence2_selected_images[i]))
            
            img1 = os.path.join (output_init_folder, os.listdir(output_init_folder)[0])
            img2 = os.path.join (output_init_folder, os.listdir(output_init_folder)[1])
            
            t1 = time.time()
            inference(model_path, img1, img2, output_rife_folder, gpu, inter_frames, 30, half)
            t2 = time.time()
            print(f'Time {round(t2 - t1, 2)}')
            print('')
           
            counter += 1
            
        counter -= fade_duration
        
        
        print('')
        
        print('------------------------------------------------------------------------------')
        
        print('')
        
        print('Sequences:', sequences[sequence_number], sequences[sequence_number+1] )
        
        print('')
        
        print('------------------------------------------------------------------------------')
        
        print('Second go')
        
        print('------------------------------------------------------------------------------')
                 
        print('counter:   ', counter)
               
        print('sequence count number:', sequence_number,'     ', sequence_number +1)
        
        print('')
        
        
            
        for i in range(fade_duration-1):

            output_init_2_folder = f'{base_rife_folder}/init2/{sequences[sequence_number]}_{sequences[sequence_number+1]}/pair{counter:05d}'
            os.makedirs(output_init_2_folder, exist_ok=True)
        
            output_rife_2_folder = f'{base_rife_folder}/rife2/{sequences[sequence_number]}_{sequences[sequence_number+1]}/pair{counter:05d}'
            os.makedirs(output_rife_2_folder, exist_ok=True)

            if i == 0 and sequence_number == 0:

                output_rife_folder = f'{base_rife_folder}/rife/{sequences[sequence_number]}_{sequences[sequence_number+1]}/pair{counter:05d}'
                shutil.copy(os.path.join(output_rife_folder, f"{i:05d}.png"),
                os.path.join(output_folder, f"{counterk:05d}.png"))
                print('Picked up first file in video: ', os.path.join(output_rife_folder, f"{i:05d}.png"))
                print('')
                counterk +=1
                counter +=1
                
            else:

                shutil.copy(os.path.join(output_folder, f"{counterk-1:05d}.png"),
                            os.path.join (output_init_2_folder, f"{counter:05d}.png"))

                print('Using file: ', os.path.join(output_folder, f"{counterk-1:05d}.png"))
            
                output_rife_folder_next = f'{base_rife_folder}/rife/{sequences[sequence_number]}_{sequences[sequence_number+1]}/pair{counter+1:05d}'

                shutil.copy(os.path.join(output_rife_folder_next, f"{i+1:05d}.png"), 
                            os.path.join(output_init_2_folder, f"{counter+1:05d}_2.png"))
                print('Using file: ', os.path.join(output_rife_folder_next, f"{i+1:05d}.png"))
    
                img1 = os.path.join (output_init_2_folder, os.listdir(output_init_2_folder)[0])
                img2 = os.path.join (output_init_2_folder, os.listdir(output_init_2_folder)[1])
    
                t1 = time.time()
                inference(model_path, img1, img2, output_rife_2_folder, gpu, inter_frames2, 30, half)
                t2 = time.time()
                print(f'Time {round(t2 - t1, 2)}')
    
                rife_intermediate_file_path = os.path.join(output_rife_2_folder, (os.listdir(output_rife_2_folder)[1]))
                print('Picked up ', rife_intermediate_file_path, 'file')
                print('')
                shutil.copy (rife_intermediate_file_path, os.path.join(output_folder, f"{counterk:05d}.png"))
            
                counterk += 1
    
                counter += 1
        
        tseq_end = time.time()
        
        time_elapsed = tseq_end - t0
        
        time_sequence = tseq_end - tseq_start
    
        time_per_frame = time_sequence/fade_duration
        
        frames_left -= 2*fade_duration 
    
        time_estimate_left = time_per_frame*frames_left/2
        
        print('------------------------------------------------------------------------------')
        print('')
        print('Sequence done!')
        print('')
        print(f'Sequence time {time_sequence:.0f} seconds')
        print('')
        print(f'Time per frame {time_per_frame:.0f} seconds')
        print('')
        print('Frames left', frames_left)
        print('')
        total_seconds = time_elapsed
        hours = total_seconds // 3600
        minutes = (total_seconds % 3600) // 60
        seconds = total_seconds % 60
        print(f"Time elapsed {hours:.0f} hours {minutes:.0f} minutes {seconds:.0f} seconds")
        print('')
        total_seconds = time_estimate_left
        hours = total_seconds // 3600
        minutes = (total_seconds % 3600) // 60
        seconds = total_seconds % 60
        print(f"Time left {hours:.0f} hours {minutes:.0f} minutes {seconds:.0f} seconds")
        print('')


        # if press_to_continue == True:
            
        #     if user_input != 'disable':

        #         user_input = input("Press Enter to continue or type 'disable' to switch the pausing off ")
        #         print('')
        try:
            shutil.rmtree(f'{base_rife_folder}/init/{sequences[sequence_number]}_{sequences[sequence_number+1]}')
            print('Deleted folder: ', f'{base_rife_folder}/init/{sequences[sequence_number]}_{sequences[sequence_number+1]}')
            print('')
        except Exception as e:
            print(f"An error occurred while deleting the folder: {e}")
            
        try:
            shutil.rmtree(f'{base_rife_folder}/rife/{sequences[sequence_number]}_{sequences[sequence_number+1]}')
            print('Deleted folder: ', f'{base_rife_folder}/rife/{sequences[sequence_number]}_{sequences[sequence_number+1]}')
            print('')
        except Exception as e:
            print(f"An error occurred while deleting the folder: {e}")
        
        try:
            shutil.rmtree(f'{base_rife_folder}/init2/{sequences[sequence_number]}_{sequences[sequence_number+1]}')
            print('Deleted folder: ', f'{base_rife_folder}/init2/{sequences[sequence_number]}_{sequences[sequence_number+1]}')
            print('')
        except Exception as e:
            print(f"An error occurred while deleting the folder: {e}")
        
        try:
            shutil.rmtree(f'{base_rife_folder}/rife2/{sequences[sequence_number]}_{sequences[sequence_number+1]}')
            print('Deleted folder: ', f'{base_rife_folder}/rife2/{sequences[sequence_number]}_{sequences[sequence_number+1]}')
            print('')
        except Exception as e:
            print(f"An error occurred while deleting the folder: {e}")




    elif sequence_number == len(sequences)-1:
        
        print('')
        
        print('------------------------------------------------------------------------------')
        
        print('')
        
        print('Final sequence: ', sequences[sequence_number])
        
        print('')
        
        print('------------------------------------------------------------------------------')
        
        print('counter:   ', counter)
        
        print('sequence count number:     ', sequence_number)
    
        
        
        files_list_2 = os.listdir(os.path.join(sequences_folder, sequences[sequence_number]))
        sequence2_selected_images = files_list_2[fade_duration-1:]  
        
        for i in range(len(sequence2_selected_images)-1):
        
            output_init_2_folder = f'{base_rife_folder}/init2/{sequences[sequence_number]}/pair{counter:05d}'
            os.makedirs(output_init_2_folder, exist_ok=True)
            
            output_rife_2_folder = f'{base_rife_folder}/rife2/{sequences[sequence_number]}/pair{counter:05d}'
            os.makedirs(output_rife_2_folder, exist_ok=True)
        
            last_sequence_file_path_1 = os.path.join(sequences_folder, sequences[sequence_number], sequence2_selected_images[i-1])
            
            last_sequence_file_path_2 = os.path.join(sequences_folder, sequences[sequence_number], sequence2_selected_images[i+1])
            
            if i == 0:

                shutil.copy(os.path.join(output_folder, f"{counterk-1:05d}.png"),
                            os.path.join (output_init_2_folder, f"{counter:05d}.png"))
                print('i = 0')
                print('Using file: ', os.path.join(output_folder, f"{counterk-1:05d}.png"))
                
            else:
                
                shutil.copy(os.path.join(last_sequence_file_path_1),
                            os.path.join (output_init_2_folder, f"{counter:05d}.png"))
                print('Using file: ', os.path.join(last_sequence_file_path_1))
                
            shutil.copy(os.path.join(last_sequence_file_path_2), 
                            os.path.join(output_init_2_folder, f"{counter:05d}_2.png"))
            print('Using file: ', os.path.join(last_sequence_file_path_2))

            img1 = os.path.join (output_init_2_folder, os.listdir(output_init_2_folder)[0])
            img2 = os.path.join (output_init_2_folder, os.listdir(output_init_2_folder)[1])

            t1 = time.time()
            inference(model_path, img1, img2, output_rife_2_folder, gpu, inter_frames2, 30, half)
            t2 = time.time()
            print(f'Time {round(t2 - t1, 2)}')
            print('')
         
            rife_intermediate_file_path = os.path.join(output_rife_2_folder, (os.listdir(output_rife_2_folder)[1]))
        
            shutil.copy (rife_intermediate_file_path, os.path.join(output_folder, f"{counterk:05d}.png"))
            
            counterk += 1
            
            counter += 1
            
        print('')
        print('------------------------------------------------------------------------------')
        print('')
        print('Sequence done!')
        print('')
        
time.sleep(1)
            
try:
    shutil.rmtree(base_rife_folder)
    print('Deleted folder: ', base_rife_folder)
except Exception as e:
    print(f"An error occurred while deleting the folder: {e}")

print('')            
print('Finished!')           
print('')

t3 = time.time()

time_difference = t3 - t0
total_seconds = round(t3 - t0, 0)
hours = total_seconds // 3600
minutes = (total_seconds % 3600) // 60
seconds = total_seconds % 60

# Print the formatted time
print(f"Time {hours:.0f} hours {minutes:.0f} minutes {seconds:.0f} seconds")
print('')

files = os.listdir(output_folder)

# Find the minimum and maximum file numbers
min_num = None
max_num = None

for filename in files:
    if filename.endswith('.png') and filename[:-4].isdigit():
        num = int(filename[:-4])
        if min_num is None or num < min_num:
            min_num = num
        if max_num is None or num > max_num:
            max_num = num

# Check for missing files
missing_files = []
for num in range(min_num, max_num + 1):
    filename = f'{num:05d}.png'
    if filename not in files:
        missing_files.append(filename)

# Print the result
if missing_files:
    print(f'Files missing in "{output_folder}" folder:')
    print('-------------------')
    for missing_file in missing_files:
        print(missing_file)
else:
    print(f'No missing files in "{output_folder}" folder')



Using GPU: NVIDIA GeForce RTX 4090
Using sequences:

0 out_00000 number of frames: 16
1 out_00015 number of frames: 31
2 out_00030 number of frames: 31

Total number of frames: 78

Counter set to 0

------------------------------------------------------------------------------

Sequences: out_00000 out_00015

------------------------------------------------------------------------------
First go
------------------------------------------------------------------------------
counter:    0
sequence count number: 0       1

Using file:  Stash/out_00000/00000.png
Using file:  Stash/out_00015/00000.png


/opt/conda/lib/python3.10/site-packages/torch/nn/modules/module.py:1501: UserWarning: operator() sees varying value in profiling, ignoring and this should be handled by GUARD logic (Triggered internally at /opt/conda/conda-bld/pytorch_1682343967769/work/third_party/nvfuser/csrc/parser.cpp:3777.)
  return forward_call(*args, **kwargs)
/opt/conda/lib/python3.10/site-packages/torch/nn/modules/module.py:1501: UserWarning: Using padding='same' with even kernel lengths and odd dilation may require a zero-padded copy of the input be created (Triggered internally at /opt/conda/conda-bld/pytorch_1682343967769/work/aten/src/ATen/native/Convolution.cpp:1003.)
  return forward_call(*args, **kwargs)
/opt/conda/lib/python3.10/site-packages/torch/nn/modules/module.py:1501: UserWarning: operator() profile_node %511 : int[] = prim::profile_ivalue[profile_failed="varying profile values"](%509)
 does not have profile information (Triggered internally at /opt/conda/conda-bld/pytorch_1682343967769/work/thi

Time 37.84

Using file:  Stash/out_00000/00001.png
Using file:  Stash/out_00015/00001.png
Time 28.41

Using file:  Stash/out_00000/00002.png
Using file:  Stash/out_00015/00002.png
Time 28.51

Using file:  Stash/out_00000/00003.png
Using file:  Stash/out_00015/00003.png
Time 27.9

Using file:  Stash/out_00000/00004.png
Using file:  Stash/out_00015/00004.png
Time 28.02

Using file:  Stash/out_00000/00005.png
Using file:  Stash/out_00015/00005.png
Time 27.92

Using file:  Stash/out_00000/00006.png
Using file:  Stash/out_00015/00006.png
Time 27.83

Using file:  Stash/out_00000/00007.png
Using file:  Stash/out_00015/00007.png
Time 27.93

Using file:  Stash/out_00000/00008.png
Using file:  Stash/out_00015/00008.png
Time 27.76

Using file:  Stash/out_00000/00009.png
Using file:  Stash/out_00015/00009.png
Time 27.7

Using file:  Stash/out_00000/00010.png
Using file:  Stash/out_00015/00010.png
Time 27.84

Using file:  Stash/out_00000/00011.png
Using file:  Stash/out_00015/00011.png
Time 27.48

